In [1]:
import sys
sys.path.insert(0, "../..")

import torch
import torch.nn as nn
from src.data import make_dataset
from pathlib import Path
from loguru import logger

/home/vscode/.cache/pypoetry/virtualenvs/deep-learning-jHmOY0S3-py3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Lets start with our good'ol MNIST.

In [3]:
datadir = Path("../../data/raw/")
batch_size = 64
train_dataloader, test_dataloader = make_dataset.get_MNIST(datadir, batch_size=batch_size) 

In [4]:
datadir.resolve().exists()

True

In [5]:
len(train_dataloader), len(test_dataloader)

(938, 157)

We can obtain an item:

In [6]:
x, y = next(iter(train_dataloader))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

The image follows the channels-first convention: (channel, width, height). The label is an integer.

Let's re-use the model we had:

In [7]:
import torch
from torch import nn

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self, filters, units1, units2, input_size=(32, 1, 28, 28)):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filters, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        activation_map_size = self._conv_test(input_size)
        logger.info(f"Aggregating activationmap with size {activation_map_size}")
        self.agg = nn.AvgPool2d(activation_map_size)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, 10)
        )

    def _conv_test(self, input_size = (32, 1, 28, 28)):
        x = torch.ones(input_size)
        x = self.convolutions(x)
        return x.shape[-2:]

    def forward(self, x):
        x = self.convolutions(x)
        x = self.agg(x)
        logits = self.dense(x)
        return logits

model = CNN(filters=32, units1=128, units2=64).to(device)

2023-05-09 10:35:29.993 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])


Using cpu device


In [8]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           9,248
              ReLU-5           [-1, 32, 12, 12]               0
         MaxPool2d-6             [-1, 32, 6, 6]               0
            Conv2d-7             [-1, 32, 4, 4]           9,248
              ReLU-8             [-1, 32, 4, 4]               0
         MaxPool2d-9             [-1, 32, 2, 2]               0
        AvgPool2d-10             [-1, 32, 1, 1]               0
          Flatten-11                   [-1, 32]               0
           Linear-12                  [-1, 128]           4,224
             ReLU-13                  [-1, 128]               0
           Linear-14                   

And set up the optimizer, loss and accuracy.

In [9]:
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [10]:
yhat = model(x)
accuracy(y, yhat)

tensor(0.0781)

# MLflow
MLflow is an open-source platform designed to manage the entire Machine Learning (ML) lifecycle, including experimentation, reproducibility, deployment, and governance. It provides a set of APIs and tools to streamline ML workflows, making it easier to track experiments, package code, manage model versions, and deploy models.

Reasons to use MLflow over TensorBoard, gin-config, or Ray:

- End-to-end ML lifecycle management: While TensorBoard focuses on visualizing model training metrics and gin-config on hyperparameter configuration, MLflow covers a broader range of tasks, such as experiment tracking, model packaging, and deployment.

- Framework agnostic: MLflow is not tied to a specific ML framework, making it suitable for projects using different libraries or even multiple libraries.

- Model Registry: MLflow provides a centralized model registry, allowing you to version, track, and manage your models, which is not available in TensorBoard or gin-config.

- Deployment support: MLflow facilitates model deployment to various platforms, such as local, cloud, or Kubernetes environments, whereas TensorBoard and gin-config are not built for deployment tasks.

- Integration with other tools: MLflow integrates with popular tools and platforms like Databricks, AWS, and Azure, making it easy to incorporate into existing workflows.

However, the choice between MLflow and other tools like TensorBoard, gin-config, or Ray depends on your specific use case and the scope of the ML workflow you want to manage.

In [12]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mnist_convolutions")

<Experiment: artifact_location='/Users/rgrouls/code/ML22/notebooks/2_convolutions/mlruns/1', creation_time=1682071009024, experiment_id='1', last_update_time=1682071009024, lifecycle_stage='active', name='mnist_convolutions', tags={}>

In the code above, we set the MLflow tracking URI to a local SQLite database file. This is done to configure the storage location for MLflow's experiment tracking data, such as metrics, parameters, and artifacts. By specifying a SQLite database, we enable a lightweight and easy-to-use storage solution for tracking the experiments and their associated information.

The line mlflow.set_experiment("mnist_convolutions") sets the active MLflow experiment to "mnist_convolutions". This is useful for organizing and grouping your runs, as it allows you to associate the upcoming ML training runs with a specific experiment name, making it easier to search, compare, and analyze the results later.

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

We import functions and classes from the hyperopt library to perform hyperparameter optimization. This library helps us find the best hyperparameter values for our machine learning model by searching through a defined search space and using optimization algorithms like Tree-structured Parzen Estimator (TPE). The goal is to improve our model's performance by tuning its hyperparameters.

Advantages of TPE:

- Model-based approach: TPE is a Bayesian optimization method that models the objective function as a probability distribution. It learns from previous evaluations to decide which points in the search space to explore next, making it more efficient in finding optimal hyperparameters.

- Exploration-exploitation trade-off: TPE balances the trade-off between exploration (searching in new regions of the search space) and exploitation (refining around the current best points). This can lead to better results in problems with complex search spaces.

- Continuous hyperparameter optimization: TPE can handle continuous hyperparameters more naturally, as it builds a probability model to estimate the performance for any given point in the search space.

Lets set up an objective function and start logging some usefull things we might want to track:

In [14]:
import torch.optim as optim
from src.models import metrics
from src.models import train_model
from datetime import datetime
modeldir = Path("./models")

# Define the objective function for hyperparameter optimization
def objective(params):
    # Start a new MLflow run for tracking the experiment
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", "convnet")
        mlflow.set_tag("dev", "raoul")
        # Log hyperparameters to MLflow
        mlflow.log_params(params)
        mlflow.log_param("datadir", f"{datadir.resolve()}")
        mlflow.log_param("batchsize", f"{batch_size}")

        # Initialize the optimizer, loss function, and accuracy metric      
        optimizer = optim.Adam
        loss_fn = torch.nn.CrossEntropyLoss()
        accuracy = metrics.Accuracy()

        # Instantiate the CNN model with the given hyperparameters
        model = CNN(**params)
        # Train the model using a custom train loop
        model, test_loss = train_model.trainloop(
            epochs=3,
            model=model,
            optimizer=optimizer,
            learning_rate=1e-3,
            loss_fn=loss_fn,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            log_dir="modellog",
            # set the tunewriter to mlflow.
            tunewriter=["mlflow"],
            train_steps=100, #len(train_dataloader),
            eval_steps=100, #len(test_dataloader),
        )

        # Save the trained model with a timestamp   
        tag = datetime.now().strftime("%Y%m%d-%H%M")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

        # Log the saved model as an artifact in MLflow
        mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")
        return {'loss' : test_loss, 'status': STATUS_OK}

In [15]:
search_space = {
    'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
    'units1' : scope.int(hp.quniform('units1', 32, 128, 8)),
    'units2' : scope.int(hp.quniform('units2', 32, 128, 8)),
}

We define a search space for hyperparameter optimization using Hyperopt. The search space specifies the range and distribution of hyperparameters to explore during the optimization process. This is crucial for finding the optimal set of hyperparameters that yield the best performance for the machine learning model. The search space defined here includes the number of filters in the convolutional layers, and the number of units in two fully connected layers, allowing Hyperopt to find the best combination within the given ranges.


Now, finally, let us perform the hyperparameter search using the fmin function from hyperopt. The function takes the following arguments:

- `fn=objective`: The objective function to minimize, which is defined earlier to train the model and return the test loss.
- `space=search_space`: The search space defined earlier, containing the range of hyperparameters to explore.
- `algo=tpe.suggest`: The optimization algorithm to use, in this case, the Tree-structured Parzen Estimator (TPE) method.
- `max_evals=10`: The maximum number of function evaluations, i.e., the maximum number of hyperparameter combinations to try.
- `trials=Trials()`: A Trials object to store the results of each evaluation.

The fmin function searches for the best hyperparameters within the given search space using the TPE algorithm, aiming to minimize the objective function (test loss). Once the optimization process is completed, the best hyperparameters found are stored in the best_result variable.

In [17]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2023-04-23 07:48:14.411 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-04-23 07:48:14.418 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230423-0748
100%|##########| 100/100 [00:06<00:00, 14.95it/s]
2023-04-23 07:48:23.826 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.8273 test 1.2165 metric ['0.5339']
100%|##########| 100/100 [00:07<00:00, 14.26it/s]
2023-04-23 07:48:33.173 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 1.0424 test 0.9043 metric ['0.6308']
100%|##########| 100/100 [00:06<00:00, 15.13it/s]
2023-04-23 07:48:42.489 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.8456 test 0.7718 metric ['0.6991']
100%|##########| 3/3 [00:28<00:00,  9.35s/it]


 10%|█         | 1/10 [00:28<04:15, 28.42s/trial, best loss: 0.7718003863096237]

2023-04-23 07:48:42.637 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-04-23 07:48:42.640 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230423-0748
100%|##########| 100/100 [00:23<00:00,  4.27it/s]
2023-04-23 07:49:15.021 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.5531 test 1.0483 metric ['0.6236']
100%|##########| 100/100 [00:23<00:00,  4.23it/s]
2023-04-23 07:49:46.264 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.9120 test 0.8541 metric ['0.7061']
100%|##########| 100/100 [00:23<00:00,  4.23it/s]
2023-04-23 07:50:17.082 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.7766 test 0.8305 metric ['0.6777']
100%|##########| 3/3 [01:34<00:00, 31.48s/it]


 20%|██        | 2/10 [02:02<08:58, 67.33s/trial, best loss: 0.7718003863096237]

...

 80%|████████  | 8/10 [06:48<01:41, 50.63s/trial, best loss: 0.7298965337872505]

2023-04-23 07:55:03.136 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-04-23 07:55:03.139 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230423-0755
100%|##########| 100/100 [00:13<00:00,  7.17it/s]
2023-04-23 07:55:21.608 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.4423 test 0.9760 metric ['0.5872']
100%|##########| 100/100 [00:13<00:00,  7.40it/s]
2023-04-23 07:55:39.613 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.7966 test 0.8144 metric ['0.6911']
100%|##########| 100/100 [00:13<00:00,  7.43it/s]
2023-04-23 07:55:57.595 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.7249 test 0.7008 metric ['0.7481']
100%|##########| 3/3 [00:54<00:00, 18.15s/it]


 90%|█████████ | 9/10 [07:43<00:51, 51.86s/trial, best loss: 0.700791663825512] 

2023-04-23 07:55:57.667 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-04-23 07:55:57.670 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230423-0755
100%|##########| 100/100 [00:07<00:00, 13.63it/s]
2023-04-23 07:56:07.540 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.5972 test 0.9853 metric ['0.6311']
100%|##########| 100/100 [00:07<00:00, 13.63it/s]
2023-04-23 07:56:17.381 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.9484 test 0.8170 metric ['0.7044']
100%|##########| 100/100 [00:07<00:00, 13.42it/s]
2023-04-23 07:56:27.326 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.7747 test 0.7585 metric ['0.7270']
100%|##########| 3/3 [00:29<00:00,  9.88s/it]


100%|██████████| 10/10 [08:13<00:00, 49.32s/trial, best loss: 0.700791663825512]


After running this, you can look at the best_result

In [15]:
best_result

{'filters': 112.0, 'units1': 72.0, 'units2': 96.0}

But you can also explore the UI from mlflow. It is pretty nice. The help you out, you can use the makefile by typing `make` in the terminal.